# Training and Finetuning ChatGPT 3.5 model for Cypher to ArangoDB conversion

## Prerequisites: 
1. OpenAI API key please set in your environment as OPENAI_API_KEY
2. Sample queries for Cypher Translated to AQL to train on (This notebook contains some samples, but they will vary depending upon your use case)

## Training / Tuning Dataset
This is the training data to fine tune the model. The format is as follows:
* desc_X : Description of the query
* cypher_ex_X : Example of Cypher query
* aql_tr_X: Translated AQL query for the Cypher query

You can potentially read these and your own examples from a file. Initially you may have to manually or in a semi-automated fashion (using Chat GPT) convert your Cypher queries to AQL queries. As your training/fine tuning gets better , the less manual intervention would be required for your query

In [1]:
# Find all friends of a person named "Alice" and return their names
dataset = {
"desc_1" :"Find all friends of a person named Alice and return their names",
    
"cypher_ex_1" : """
MATCH (p:Person)-[:FRIEND]->(f:Person)
WHERE p.name = 'Alice'
RETURN p.name, f.name
""",

"aql_tr_1": """
FOR p IN Person
    FOR f IN 1..1 OUTBOUND p FRIEND
    FILTER p.name == 'Alice'
    RETURN { pName: p.name, fName: f.name }
""",

"desc_2": "Find Movies Directed by a Specific Director",

"cypher_ex_2": """
MATCH (d:Director)-[:DIRECTED]->(m:Movie)
WHERE d.name = 'Steven Spielberg'
RETURN m.title
""",
    
"aql_tr_2": """
FOR d IN Director
    FOR m IN 1..1 OUTBOUND d DIRECTED
    FILTER d.name == 'Steven Spielberg'
    RETURN m.title
""",

"desc_3": "Find Employees Managed by a Specific Manager",

"cypher_ex_3": """
MATCH (manager:Employee)-[:MANAGES]->(employee:Employee)
WHERE manager.name = 'John Doe'
RETURN employee.name
""",
    
"aql_tr_3": """
FOR manager IN Employee
    FOR employee IN 1..1 OUTBOUND manager MANAGES
    FILTER manager.name == 'John Doe'
    RETURN employee.name
""",

"desc_4": "Find All Authors of a Book",

"cypher_ex_4": """
MATCH (a:Author)-[:WROTE]->(b:Book)
WHERE b.title = 'The Great Gatsby'
RETURN a.name
""",
    
"aql_tr_4": """
FOR a IN Author
    FOR b IN 1..1 OUTBOUND a WROTE
    FILTER b.title == 'The Great Gatsby'
    RETURN a.name
""",
    

"desc_5": "Find All Actors Who Starred in a Movie",
    
"cypher_ex_5": """
MATCH (a:Actor)-[:ACTED_IN]->(m:Movie)
WHERE m.title = 'Inception'
RETURN a.name
""",
    
"aql_tr_5": """
FOR a IN Actor
    FOR m IN 1..1 OUTBOUND a ACTED_IN
    FILTER m.title == 'Inception'
    RETURN a.name
""",


"desc_6": "Find All Products Purchased by a Customer",
    
"cypher_ex_6": """
MATCH (c:Customer)-[:PURCHASED]->(p:Product)
WHERE c.username = 'johndoe'
RETURN p.name
""",
    
"aql_tr_6": """
FOR c IN Customer
    FOR p IN 1..1 OUTBOUND c PURCHASED
    FILTER c.username == 'johndoe'
    RETURN p.name
""",
    

"desc_7": "Find All Cities Connected to a Specific Airport",
    
"cypher_ex_7": """
MATCH (a:Airport)-[:CONNECTS_TO]->(c:City)
WHERE a.code = 'JFK'
RETURN c.name
""",
    
"aql_tr_7": """
FOR a IN Airport
    FOR c IN 1..1 OUTBOUND a CONNECTS_TO
    FILTER a.code == 'JFK'
    RETURN c.name
""",
    

"desc_8": "Find All Products in a Specific Category",
    
"cypher_ex_8": """
MATCH (c:Category)<-[:IN_CATEGORY]-(p:Product)
WHERE c.name = 'Electronics'
RETURN p.name
""",
    
"aql_tr_8": """
FOR c IN Category
    FOR p IN 1..1 INBOUND c IN_CATEGORY
    FILTER c.name == 'Electronics'
    RETURN p.name
""",
    
"desc_9": "Find All Courses Taught by a Specific Professor",
    
"cypher_ex_9": """
MATCH (p:Professor)-[:TEACHES]->(c:Course)
WHERE p.name = 'Dr. Smith'
RETURN c.title
""",
    
"aql_tr_9": """
FOR p IN Professor
    FOR c IN 1..1 OUTBOUND p TEACHES
    FILTER p.name == 'Dr. Smith'
    RETURN c.title
""",
    
"desc_10": "Find All Ingredients Used in a Recipe",

"cypher_ex_10": """
MATCH (r:Recipe)-[:USES]->(i:Ingredient)
WHERE r.name = 'Spaghetti Carbonara'
RETURN i.name
""",
    
"aql_tr_10": """
FOR r IN Recipe
    FOR i IN 1..1 OUTBOUND r USES
    FILTER r.name == 'Spaghetti Carbonara'
    RETURN i.name
""",
    
"desc_11": "Find All Students Enrolled in a Course",

"cypher_ex_11": """
MATCH (s:Student)-[:ENROLLED_IN]->(c:Course)
WHERE c.code = 'CS101'
RETURN s.name
""",
    
"aql_tr_11": """
FOR s IN Student
    FOR c IN 1..1 OUTBOUND s ENROLLED_IN
    FILTER c.code == 'CS101'
    RETURN s.name
""",
    
"desc_12":"Find All Books Written by an Author",

"cypher_ex_12": """
MATCH (a:Author)-[:WROTE]->(b:Book)
WHERE a.name = 'J.K. Rowling'
RETURN b.title
""",
    
"aql_tr_12": """
FOR a IN Author
    FOR b IN 1..1 OUTBOUND a WROTE
    FILTER a.name == 'J.K. Rowling'
    RETURN b.title
""",
    
"desc_13": "Find All Employees Reporting to a Manager",
    
"cypher_ex_13": """
MATCH (manager:Employee)-[:MANAGES]->(employee:Employee)
WHERE manager.name = 'Jane Smith'
RETURN employee.name
""",
    
"aql_tr_13": """
FOR manager IN Employee
    FOR employee IN 1..1 OUTBOUND manager MANAGES
    FILTER manager.name == 'Jane Smith'
    RETURN employee.name
""",
    
"desc_14": "Find All Movies Where Two Actors Co-Starred",
    
"cypher_ex_14": """
MATCH (a1:Actor)-[:ACTED_IN]->(m:Movie)<-[:ACTED_IN]-(a2:Actor)
WHERE a1.name = 'Tom Hanks' AND a2.name = 'Meg Ryan'
RETURN m.title
""",
    
"aql_tr_14": """
FOR a1 IN Actor
    FOR m IN 1..1 OUTBOUND a1 ACTED_IN
        FOR a2 IN 1..1 INBOUND m ACTED_IN
        FILTER a1.name == 'Tom Hanks' AND a2.name == 'Meg Ryan'
        RETURN m.title
""",
    
"desc_15": "Find All Products Purchased by Frequent Customers",
    
"cypher_ex_15": """
MATCH (c:Customer)-[:PURCHASED]->(p:Product)
WHERE c.total_purchases > 1000
RETURN c.name, p.name
""",
    
"aql_tr_15": """
FOR c IN Customer
    FILTER c.total_purchases > 1000
    FOR p IN 1..1 OUTBOUND c PURCHASED
    RETURN { customer: c.name, product: p.name }
""",
    
"desc_16": "Find All Airports Connected to a City by a Specific Airline",
    
"cypher_ex_16": """
MATCH (c:City)<-[:LOCATED_IN]-(a:Airport)-[:OPERATES_BY]->(airline:Airline)
WHERE c.name = 'New York' AND airline.name = 'Delta Airlines'
RETURN a.name
""",
    
"aql_tr_16": """
FOR c IN City
    FILTER c.name == 'New York'
    FOR a IN 1..1 INBOUND c LOCATED_IN
        FOR airline IN 1..1 OUTBOUND a OPERATES_BY
        FILTER airline.name == 'Delta Airlines'
        RETURN a.name
""",
    
"desc_17": "Find All Customers Who Bought Products from a Specific Category",
    
"cypher_ex_17": """
MATCH (c:Customer)-[:PURCHASED]->(p:Product)-[:IN_CATEGORY]->(cat:Category)
WHERE cat.name = 'Electronics'
RETURN c.name, p.name
""",
    
"aql_tr_17": """
FOR c IN Customer
    FOR p IN 1..1 OUTBOUND c PURCHASED
        FOR cat IN 1..1 INBOUND p IN_CATEGORY
        FILTER cat.name == 'Electronics'
        RETURN { customer: c.name, product: p.name }
""",
    
"desc_18": "Find All Professors Teaching Multiple Courses",
    
"cypher_ex_18": """
MATCH (p:Professor)-[:TEACHES]->(c:Course)
WITH p, COUNT(c) AS courseCount
WHERE courseCount > 1
RETURN p.name, courseCount
""",
    
"aql_tr_18": """
FOR p IN Professor
    FOR c IN 1..1 OUTBOUND p TEACHES
    COLLECT p, courseCount = LENGTH(c)
    FILTER courseCount > 1
    RETURN { professor: p.name, courseCount: courseCount }
""",
    
"desc_19": "Find All Cities Connected to a Specific City by Multiple Airlines",
    
"cypher_ex_19": """
MATCH (c1:City)-[:CONNECTS_TO]->(c2:City)<-[:CONNECTS_TO]-(a:Airport)-[:OPERATES_BY]->(airline:Airline)
WHERE c1.name = 'Los Angeles'
WITH c2, airline, COUNT(DISTINCT airline) AS airlineCount
WHERE airlineCount > 1
RETURN c2.name, airlineCount
""",
    
"aql_tr_19": """
FOR c1 IN City
    FILTER c1.name == 'Los Angeles'
    FOR c2 IN 1..1 OUTBOUND c1 CONNECTS_TO
        FOR a IN 1..1 INBOUND c2 CONNECTS_TO
            FOR airline IN 1..1 OUTBOUND a OPERATES_BY
            COLLECT c2, airlineCount = LENGTH(DISTINCT airline)
            FILTER airlineCount > 1
            RETURN { city: c2.name, airlineCount: airlineCount }
""",
    
"desc_20": "Find All Courses That Share Students",
    
"cypher_ex_20": """
MATCH (s:Student)-[:ENROLLED_IN]->(c1:Course)<-[:ENROLLED_IN]-(s2:Student)-[:ENROLLED_IN]->(c2:Course)
WHERE c1 <> c2
RETURN s.name, c1.title, c2.title
""",
    
"aql_tr_20": """
FOR s IN Student
    FOR c1 IN 1..1 OUTBOUND s ENROLLED_IN
        FOR s2 IN 1..1 INBOUND c1 ENROLLED_IN
            FOR c2 IN 1..1 OUTBOUND s2 ENROLLED_IN
            FILTER c1 != c2
            RETURN { student: s.name, course1: c1.title, course2: c2.title }
""",
    
"desc_21": "Find All Companies and Their Employees in a Specific City",
    
"cypher_ex_21": """
MATCH (c:Company)-[:EMPLOYS]->(e:Employee)-[:LOCATED_IN]->(city:City)
WHERE city.name = 'San Francisco'
RETURN c.name, e.name
""",
    
"aql_tr_21": """
FOR c IN Company
    FOR e IN 1..1 OUTBOUND c EMPLOYS
        FOR city IN 1..1 INBOUND e LOCATED_IN
        FILTER city.name == 'San Francisco'
        RETURN { company: c.name, employee: e.name }
""",
    
"desc_22": "Find All Authors Who Co-Wrote a Book",
    
"cypher_ex_22": """
MATCH (a1:Author)-[:WROTE]->(b:Book)<-[:WROTE]-(a2:Author)
WHERE a1 <> a2
RETURN a1.name, a2.name, b.title
""",
    
"aql_tr_22": """
FOR a1 IN Author
    FOR b IN 1..1 OUTBOUND a1 WROTE
        FOR a2 IN 1..1 OUTBOUND b WROTE
        FILTER a1 != a2
        RETURN { author1: a1.name, author2: a2.name, book: b.title }
""",
    
"desc_23": "Find All Movies Directed by Actors",
    
"cypher_ex_23": """
MATCH (a:Actor)-[:DIRECTED]->(m:Movie)
RETURN a.name, m.title
""",
    
"aql_tr_23": """
FOR a IN Actor
    FOR m IN 1..1 OUTBOUND a DIRECTED
    RETURN { actor: a.name, movie: m.title }
""",
    
"desc_24": "Find All Customers Who Purchased the Same Product",
    
"cypher_ex_24": """
MATCH (c1:Customer)-[:PURCHASED]->(p:Product)<-[:PURCHASED]-(c2:Customer)
WHERE c1 <> c2
RETURN c1.name, c2.name, p.name
""",
    
"aql_tr_24": """
FOR c1 IN Customer
    FOR p IN 1..1 OUTBOUND c1 PURCHASED
        FOR c2 IN 1..1 INBOUND p PURCHASED
        FILTER c1 != c2
        RETURN { customer1: c1.name, customer2: c2.name, product: p.name }
""",
    
"desc_25": "Find All Airports That Share the Same City",
    
"cypher_ex_25": """
MATCH (a1:Airport)-[:LOCATED_IN]->(city:City)<-[:LOCATED_IN]-(a2:Airport)
WHERE a1 <> a2
RETURN a1.name, a2.name, city.name
""",
    
"aql_tr_25": """
FOR a1 IN Airport
    FOR city IN 1..1 INBOUND a1 LOCATED_IN
        FOR a2 IN 1..1 OUTBOUND city LOCATED_IN
        FILTER a1 != a2
        RETURN { airport1: a1.name, airport2: a2.name, city: city.name }
""",
    
"desc_26":"Find All Professors Who Teach Both Undergraduate and Graduate Courses",
    
"cypher_ex_26": """
MATCH (p:Professor)-[:TEACHES]->(c1:Course { level: 'Undergraduate' })<-
[:TEACHES]-(p:Professor)-[:TEACHES]->(c2:Course { level: 'Graduate' })
RETURN p.name, c1.title, c2.title
""",
    
"aql_tr_26": """
FOR p IN Professor
    FOR c1 IN 1..1 OUTBOUND p TEACHES
        FOR c2 IN 1..1 OUTBOUND p TEACHES
        FILTER c1 != c2 AND c1.level == 'Undergraduate' AND c2.level == 'Graduate'
        RETURN { professor: p.name, course1: c1.title, course2: c2.title }
""",
    
"desc_27": "Find All Actors Who Have Acted in Movies of a Specific Genre",
    
"cypher_ex_27": """
MATCH (a:Actor)-[:ACTED_IN]->(m:Movie)-[:IN_GENRE]->(g:Genre)
WHERE g.name = 'Action'
RETURN a.name, m.title
""",
    
"aql_tr_27": """
FOR a IN Actor
    FOR m IN 1..1 OUTBOUND a ACTED_IN
        FOR g IN 1..1 OUTBOUND m IN_GENRE
        FILTER g.name == 'Action'
        RETURN { actor: a.name, movie: m.title }
""",
    
"desc_28": "Find All Customers Who Purchased Products in Multiple Categories",
    
"cypher_ex_28": """
MATCH (c:Customer)-[:PURCHASED]->(p1:Product)-[:IN_CATEGORY]->(cat1:Category)<-[:IN_CATEGORY]-(p2:Product)-[:PURCHASED]-(c:Customer)
WHERE p1 <> p2
RETURN c.name, cat1.name
""",
    
"aql_tr_28": """
FOR c IN Customer
    FOR p1 IN 1..1 OUTBOUND c PURCHASED
        FOR cat1 IN 1..1 INBOUND p1 IN_CATEGORY
            FOR p2 IN 1..1 OUTBOUND c PURCHASED
            FOR cat2 IN 1..1 INBOUND p2 IN_CATEGORY
            FILTER p1 != p2 AND cat1.name != cat2.name
            RETURN { customer: c.name, category1: cat1.name, category2: cat2.name }
""",
    
"desc_29": "Find All Employees Who Supervise Other Employees",
    
"cypher_ex_29": """
MATCH (manager:Employee)-[:MANAGES]->(employee:Employee)
RETURN manager.name, employee.name
""",
    
"aql_tr_29": """
FOR manager IN Employee
    FOR employee IN 1..1 OUTBOUND manager MANAGES
    RETURN { manager: manager.name, employee: employee.name }
""",
    
"desc_30": "Find All Employees Who Share the Same Manager",
    
"cypher_ex_30": """
MATCH (manager:Employee)-[:MANAGES]->(employee1:Employee)<-[:MANAGES]-(employee2:Employee)
WHERE employee1 <> employee2
RETURN manager.name, employee1.name, employee2.name
""",
    
"aql_tr_30": """
FOR manager IN Employee
    FOR employee1 IN 1..1 OUTBOUND manager MANAGES
        FOR employee2 IN 1..1 INBOUND manager MANAGES
        FILTER employee1 != employee2
        RETURN { manager: manager.name, employee1: employee1.name, employee2: employee2.name }
""",
    
"desc_31": "Find All Authors Who Have Co-Authored Multiple Books",
    
"cypher_ex_31": """
MATCH (a1:Author)-[:WROTE]->(b1:Book)<-[:WROTE]-(a2:Author)-[:WROTE]->(b2:Book)
WHERE a1 <> a2 AND b1 <> b2
RETURN a1.name, a2.name, b1.title, b2.title
""",
    
"aql_tr_31": """
FOR a1 IN Author
    FOR b1 IN 1..1 OUTBOUND a1 WROTE
        FOR a2 IN 1..1 OUTBOUND b1 WROTE
            FOR b2 IN 1..1 INBOUND a2 WROTE
            FILTER a1 != a2 AND b1 != b2
            RETURN { author1: a1.name, author2: a2.name, book1: b1.title, book2: b2.title }
""",
    
"desc_32": "Find All Students Who Share Courses With Another Student",
    
"cypher_ex_32":  """
MATCH (s1:Student)-[:ENROLLED_IN]->(c:Course)<-[:ENROLLED_IN]-(s2:Student)
WHERE s1 <> s2
RETURN s1.name, s2.name, c.title
""",
    
"aql_tr_32": """
FOR s1 IN Student
    FOR c IN 1..1 OUTBOUND s1 ENROLLED_IN
        FOR s2 IN 1..1 INBOUND c ENROLLED_IN
        FILTER s1 != s2
        RETURN { student1: s1.name, student2: s2.name, course: c.title }
"""
}

In [2]:
import tiktoken
import json
import numpy as np
from collections import defaultdict
import time

In [3]:
encoding = tiktoken.get_encoding("cl100k_base")
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")


In [4]:
# create json dataset
# store dataset.jsonl file in current working directory
with open("dataset.jsonl", "w") as f:
    for idx in range(1, 32):
        cy_k = f"cypher_ex_{idx}"
        aq_k = f"aql_tr_{idx}"
        prompt_dict = {"messages": [
        {"role": "system", "content": "Marvin is an AI chatbot that specializes in translating cypher query to ArangoDB's AQL query"},
        {"role": "user", "content":f"Translate the following cypher query {dataset[cy_k]} to ArangoDB AQL query"},
        {"role": "assistant", "content":f"Equivalent ArangoDB AQL query is {dataset[aq_k]}"}]}
        f.write(json.dumps(prompt_dict) + "\n")
    

In [5]:
# count tokens
len(encoding.encode(dataset["cypher_ex_1"] + dataset["aql_tr_1"]))


70

# Load the data 

Here we load the data for fine tuning the model with the examples

In [6]:
with open("dataset.jsonl", 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

In [7]:
# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 31
First example:
{'role': 'system', 'content': "Marvin is an AI chatbot that specializes in translating cypher query to ArangoDB's AQL query"}
{'role': 'user', 'content': "Translate the following cypher query \nMATCH (p:Person)-[:FRIEND]->(f:Person)\nWHERE p.name = 'Alice'\nRETURN p.name, f.name\n to ArangoDB AQL query"}
{'role': 'assistant', 'content': "Equivalent ArangoDB AQL query is \nFOR p IN Person\n    FOR f IN 1..1 OUTBOUND p FRIEND\n    FILTER p.name == 'Alice'\n    RETURN { pName: p.name, fName: f.name }\n"}


In [8]:
# Format error checks
format_errors = defaultdict(int)

In [9]:
for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        if not content or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

In [10]:
if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [11]:
encoding = tiktoken.get_encoding("cl100k_base")

def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

In [12]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))
    
print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 114, 243
mean / median: 153.48387096774192, 137.0
p5 / p95: 120.0, 210.0

#### Distribution of num_assistant_tokens_per_example:
min / max: 41, 116
mean / median: 62.41935483870968, 53.0
p5 / p95: 42.0, 97.0

0 examples may be over the 4096 token limit, they will be truncated during fine-tuning


In [13]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")
print(f"Esitmated charging price: {n_epochs * n_billing_tokens_in_dataset * 0.001 * 0.0080}$")

Dataset has ~4758 tokens that will be charged for during training
By default, you'll train for 3 epochs on this dataset
By default, you'll be charged for ~14274 tokens
Esitmated charging price: 0.11419200000000002$


## Fine Tuning GPT-3.5 with our own Dataset

In [14]:
import os
import openai
import gradio

In [15]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if OPENAI_API_KEY is None:
    print("Fatal Error! need API key to proceed")
else:
    openai.api_key = OPENAI_API_KEY

In [16]:
from openai import OpenAI

client = OpenAI()

file = client.files.create(
  file=open("dataset.jsonl", "rb"),
  purpose='fine-tune'
)

In [17]:
file

FileObject(id='file-KHlQd6ZxfaF8XZplpuWFDMFF', bytes=18777, created_at=1713994020, filename='dataset.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [18]:
# starts fine tuning
job = client.fine_tuning.jobs.create(training_file=file.id, model="gpt-3.5-turbo")

In [19]:
# Cancel a job
#openai.FineTuningJob.cancel(job.id)

In [20]:
# List all jobs
# for job in client.fine_tuning.jobs.list():
#     print(job)

In [21]:
# Retrieve the state of a fine-tune
jobstatus = openai.fine_tuning.jobs.retrieve(job.id)
print(f"\nJob status: {jobstatus}")
SLEEP_TIME=0.5*60


Job status: FineTuningJob(id='ftjob-FcthOU1lChzlmKV9Axbq2HRg', created_at=1713994024, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-qxPkNPWF8DKL1WPIc5huNbv1', result_files=[], seed=1176583671, status='validating_files', trained_tokens=None, training_file='file-KHlQd6ZxfaF8XZplpuWFDMFF', validation_file=None, integrations=[], user_provided_suffix=None)


In [23]:

while jobstatus.finished_at is None and jobstatus.status in ["validating_files","running"]:
    print(f"\nJob status: {jobstatus}")
    print(f"\nSleeping {SLEEP_TIME} more seconds...")
    time.sleep(SLEEP_TIME) # Sleep SLEEP_TIME seconds
    jobstatus = openai.fine_tuning.jobs.retrieve(job.id)


if jobstatus.error.code is not None or jobstatus.status != "succeeded":
    print(f"Fatal Error: {jobstatus},\nFine-tuning job failed")
    raise SystemExit("Fatal error")


timetaken= jobstatus.finished_at - jobstatus.created_at
    
print(f"Fine tuning took {timetaken/60:.2f} minutes, status {jobstatus.status}")
print(f"{jobstatus}")
    


Job status: FineTuningJob(id='ftjob-FcthOU1lChzlmKV9Axbq2HRg', created_at=1713994024, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-qxPkNPWF8DKL1WPIc5huNbv1', result_files=[], seed=1176583671, status='validating_files', trained_tokens=None, training_file='file-KHlQd6ZxfaF8XZplpuWFDMFF', validation_file=None, integrations=[], user_provided_suffix=None)

Sleeping 30.0 more seconds...

Job status: FineTuningJob(id='ftjob-FcthOU1lChzlmKV9Axbq2HRg', created_at=1713994024, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-qxPkNPWF8DKL1WPIc5huNbv1', r

### Validate Fine-Tuned Model
These examples introduce new types of relationships, such as actors directing movies, customers sharing common interests, professors collaborating on research papers, and cities connected by high-speed railways.

In [24]:
# Find All Stores and Their Products in a Specific Category
# Find All Actors Who Directed Movies
# Find All Customers Who Share a Common Interest
# Find All Professors Who Collaborated on Research Papers
# Find All Cities Connected by High-Speed Railways

val_data = { 
"val_1": """
MATCH (s:Store)-[:SELLS]->(p:Product)-[:IN_CATEGORY]->(cat:Category)
WHERE cat.name = 'Electronics'
RETURN s.name, p.name
""",

"ground_truth_1": """
FOR s IN Store
    FOR p IN 1..1 OUTBOUND s SELLS
        FOR cat IN 1..1 INBOUND p IN_CATEGORY
        FILTER cat.name == 'Electronics'
        RETURN { store: s.name, product: p.name }
""",
    
"val_2": """
MATCH (a:Actor)-[:DIRECTED]->(m:Movie)
RETURN a.name, m.title
""",

"ground_truth_2": """
FOR a IN Actor
    FOR m IN 1..1 OUTBOUND a DIRECTED
    RETURN { actor: a.name, movie: m.title }
""",
    
"val_3":"""
MATCH (c1:Customer)-[:HAS_INTEREST]->(i:Interest)<-[:HAS_INTEREST]-(c2:Customer)
WHERE c1 <> c2
RETURN c1.name, c2.name, i.name
""",

"ground_truth_3": """
FOR c1 IN Customer
    FOR i IN 1..1 OUTBOUND c1 HAS_INTEREST
        FOR c2 IN 1..1 INBOUND i HAS_INTEREST
        FILTER c1 != c2
        RETURN { customer1: c1.name, customer2: c2.name, interest: i.name }
""",
    
"val_4":"""
MATCH (p1:Professor)-[:WROTE]->(r:ResearchPaper)<-[:WROTE]-(p2:Professor)
WHERE p1 <> p2
RETURN p1.name, p2.name, r.title
""",
    
"ground_truth_4": """
FOR p1 IN Professor
    FOR r IN 1..1 OUTBOUND p1 WROTE
        FOR p2 IN 1..1 OUTBOUND r WROTE
        FILTER p1 != p2
        RETURN { professor1: p1.name, professor2: p2.name, researchPaper: r.title }

""",
    
"val_5": """
MATCH (c1:City)-[:CONNECTED_BY]->(hsr:HighSpeedRailway)<-[:CONNECTED_BY]-(c2:City)
WHERE c1 <> c2
RETURN c1.name, c2.name, hsr.name
""",
    
"ground_truth_5": """
FOR c1 IN City
    FOR hsr IN 1..1 OUTBOUND c1 CONNECTED_BY
        FOR c2 IN 1..1 INBOUND hsr CONNECTED_BY
        FILTER c1 != c2
        RETURN { city1: c1.name, city2: c2.name, highSpeedRailway: hsr.name  }
"""
}

In [25]:
print(jobstatus.fine_tuned_model)

ft:gpt-3.5-turbo-0125:arangodb::9HeX9b89


In [26]:
if jobstatus.fine_tuned_model is None:
    SystemExit("Fatal unable to get fine-tuned model, tuning job failed")
    
# validate on new data
val_cypher = val_data["val_5"]
completion = client.chat.completions.create( model=jobstatus.fine_tuned_model,
  messages=[
    {"role": "system", "content": "Marvin is an AI chatbot that specializes in translating cypher query to ArangoDB's AQL query"},
    {"role": "user", "content": f"Translate the following cypher query {val_cypher} to ArangoDB AQL query"}
  ]
)

In [27]:
# result from fine-tune model
print(completion.choices[0].message.content)

Equivalent ArangoDB AQL query is 
FOR c1 IN City
    FOR hsr IN 1..1 OUTBOUND c1 CONNECTED_BY
        FOR c2 IN 1..1 INBOUND hsr CONNECTED_BY
        FILTER c1 != c2
        RETURN { city1: c1.name, city2: c2.name, railway: hsr.name }



## Creating a Chatbot Demo
This is a sample visual interface for running the queries you can always run it programmatically using the API against the model

In [28]:
def cypher_to_aql(user_input):
    messages = [{"role": "system", "content": "Marvin is an AI chatbot that specializes in translating cypher query to ArangoDB's AQL query"},]
    messages.append({"role": "user", "content": user_input})
    completion = client.chat.completions.create(
      model=jobstatus.fine_tuned_model,
      messages= messages,
    )
    reply = completion.choices[0].message.content
    return reply

In [29]:
inputs = gradio.components.Textbox(lines=7, label="Chat with Fine-Tuned GPT-3.5")
outputs = gradio.components.Textbox(lines=7,label="Fine-Tuned GPT-3.5 Reply")
demo = gradio.Interface(fn=cypher_to_aql, inputs = inputs, outputs = outputs, title = "Cypher to ArangoDB AQL Chatbot")

In [30]:
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://51af4e627d220f3ae2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
